In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split as ttsplit
from sklearn.metrics import mean_squared_error as mse

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

from IPython.display import display, HTML

import eli5

from pyastrotrader import calculate_chart, calculate_aspects, calculate_transits
from pyastrotrader.utils import create_input_json
from pyastrotrader.constants import *

from settings import *
from helpers import *

In [ ]:
StockPrices = pd.read_csv("{}.csv".format(SOURCE_FILE))
StockPrices['Counter'] = np.arange(len(StockPrices))

In [ ]:
max_counter = StockPrices['Counter'].max()

StockPrices['CorrectedDate'] = StockPrices.apply( lambda x :  correct_date(x), axis =1 )
StockPrices['PreviousStartPrice'] = StockPrices.apply( lambda x :  get_previous_stock_price(StockPrices, x, SWING_TRADE_DURATION), axis =1 )
StockPrices['FutureFinalPrice'] = StockPrices.apply( lambda x :  get_future_stock_price(StockPrices, x, max_counter, SWING_TRADE_DURATION), axis =1 )
StockPrices['PreviousStartDate'] = StockPrices.apply( lambda x :  get_previous_stock_date(StockPrices, x, SWING_TRADE_DURATION), axis =1 )
StockPrices['FutureFinalDate'] = StockPrices.apply( lambda x :  get_future_stock_date(StockPrices, x, max_counter, SWING_TRADE_DURATION), axis =1 )

StockPrices['CurrentTrend'] = StockPrices.apply(lambda x : calculate_current_trend(x), axis = 1)

StockPrices['FutureTrend'] = StockPrices.apply(lambda x : calculate_future_trend(x), axis = 1)
StockPrices['SwingStrength'] = StockPrices.apply(lambda x: calculate_swing_strenght(x), axis =1)
StockPrices['IntradayVolatility'] = StockPrices.apply(lambda x: calculate_intraday_volatility(StockPrices, x, SWING_TRADE_DURATION), axis =1)

StockPrices['FutureTrendMax'] = StockPrices.apply(lambda x : get_future_stock_max_price(StockPrices, x, max_counter, SWING_TRADE_DURATION), axis = 1)
StockPrices['FutureTrendMin'] = StockPrices.apply(lambda x : get_future_stock_min_price(StockPrices, x, max_counter, SWING_TRADE_DURATION), axis = 1)

StockPrices['IsSwing'] = StockPrices.apply(lambda x: detect_swing_trade(x, SWING_EXPECTED_VOLATILITY), axis =1)
StockPrices['IsSwing'] = StockPrices.apply(lambda x:clean_swing_trade(StockPrices, x, SWING_EXPECTED_VOLATILITY), axis =1)

StockPrices['StockIncreasedPrice'] = StockPrices.apply(lambda x:detect_price_increase(x, STAGNATION_THRESHOLD), axis =1)

StockPrices['StockDecreasedPrice'] = StockPrices.apply(lambda x:detect_price_decrease(x, STAGNATION_THRESHOLD), axis =1)

StockPrices['StockStagnated'] = StockPrices.apply(lambda x:detect_price_stagnated(x, STAGNATION_THRESHOLD), axis =1)

StockPrices['TARGET_IS_SWING'] = StockPrices['IsSwing']
StockPrices['TARGET_PRICE_INCREASE'] = StockPrices['StockIncreasedPrice']
StockPrices['TARGET_PRICE_DECREASE'] = StockPrices['StockDecreasedPrice']
StockPrices['TARGET_PRICE_STAGNATION'] = StockPrices['StockStagnated']

In [ ]:
output_excel_file='./output/{}.Analisys.xlsx'.format(ASSET_TO_CALCULATE)
StockPrices.to_excel(output_excel_file)

In [ ]:
#StockPrices[StockPrices['IsSwing'] == 1].sort_values(by=['Date'], ascending=[0]).head(20)

In [ ]:
#StockPrices[StockPrices['StockIncreasedPrice'] == 1].sort_values(by=['Date'], ascending=[0]).head(20)

In [ ]:
#StockPrices[StockPrices['StockDecreasedPrice'] == 1].sort_values(by=['Date'], ascending=[0]).head(20)

In [ ]:
swing_to_chart = []
for index, current_swing in StockPrices[StockPrices['IsSwing'] == 1].iterrows():
    swing_to_chart.append(dict(
        x0=current_swing['CorrectedDate'], 
        x1=current_swing['CorrectedDate'], 
        y0=0, 
        y1=1, 
        xref='x', 
        yref='paper',
        line_width=2))


In [ ]:
fig = go.Figure(data=[go.Candlestick(
                x=StockPrices['CorrectedDate'],
                open=StockPrices['Open'],
                high=StockPrices['High'],
                low=StockPrices['Low'],
                close=StockPrices['Price'])])
fig.update_layout(
    title="{} Detected Swing Trade Opportunities".format(ASSET_TO_CALCULATE),
    width=1000,
    height=500,
    xaxis_rangeslider_visible=False,
    shapes=swing_to_chart,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=30,
        pad=4
    ),    
)
fig.show()

In [ ]:
asset_natal_chart_input = create_input_json(NATAL_DATE, 
                                            DEFAULT_PARAMETERS, 
                                            DEFAULT_CONFIG)

asset_natal_chart = calculate_chart(asset_natal_chart_input)
dates_to_generate = list(StockPrices['CorrectedDate'])

In [ ]:
for current_date in dates_to_generate:
    chart_input = create_input_json(current_date + 'T10:00:00-03:00', 
                                      DEFAULT_PARAMETERS, 
                                      DEFAULT_CONFIG)
    charts[current_date] = calculate_chart(chart_input)
    aspects[current_date] = calculate_transits(asset_natal_chart, charts[current_date], PLANETS_TO_CALCULATE, ASPECTS_TO_CALCULATE, 4)
    aspects_transiting[current_date]= calculate_aspects(charts[current_date], PLANETS_TO_CALCULATE, ASPECTS_TO_CALCULATE, 4)   

In [ ]:
astro_columns = []

for first_planet in PLANETS_TO_CALCULATE:
    for second_planet in PLANETS_TO_CALCULATE:
        for aspect in ASPECTS_TO_CALCULATE:
            column_name="ASTRO_{}_{}_{}".format(PLANETS[first_planet],ASPECT_NAME[aspect],PLANETS[second_planet]).upper()
            astro_columns.append(column_name)
            StockPrices['StockIncreasedPrice'] = StockPrices.apply(lambda x:detect_price_increase(x, STAGNATION_THRESHOLD), axis =1)
            StockPrices[column_name] = StockPrices.apply(lambda x:is_aspected(x, first_planet, second_planet, aspect), axis =1)
            StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float', errors='coerce')
            
            if first_planet == second_planet:
                continue
                
            column_name="ASTRO_TRANSITING_{}_{}_{}".format(PLANETS[first_planet],ASPECT_NAME[aspect],PLANETS[second_planet]).upper()
            astro_columns.append(column_name)
            StockPrices[column_name] = StockPrices.apply(lambda x:is_aspected_transiting(x, first_planet, second_planet, aspect), axis =1)
            StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float', errors='coerce')                 

In [ ]:
for first_planet in PLANETS_TO_CALCULATE:
    column_name="ASTRO_{}_RETROGADE".format(PLANETS[first_planet]).upper()
    astro_columns.append(column_name)
    StockPrices[column_name] = StockPrices.apply(lambda x:is_retrograde(x, first_planet), axis =1)
    StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float',errors='coerce')

In [ ]:
booster_price_increase, score_price_increase = get_best_booster('TARGET_PRICE_INCREASE', MAX_INTERACTIONS, StockPrices, astro_columns)
print("Best Score for Price Increase Model:{}".format(score_price_increase))

booster_price_decrease, score_price_decrease = get_best_booster('TARGET_PRICE_DECREASE', MAX_INTERACTIONS, StockPrices, astro_columns)
print("Best Score for Price Decrease Model:{}".format(score_price_decrease))

booster_price_stagnation, score_price_stagnation = get_best_booster('TARGET_PRICE_STAGNATION', MAX_INTERACTIONS, StockPrices, astro_columns)
print("Best Score for Price Stagnation Model:{}".format(score_price_stagnation))

booster_swing_trade, score_swing_trade = get_best_booster('TARGET_IS_SWING', MAX_INTERACTIONS, StockPrices, astro_columns)
print("Best Score for Swing Trade Model:{}".format(score_swing_trade))

In [ ]:
relevant_features_swing_trade = sorted( ((v,k) for k,v in booster_swing_trade.get_score().items()), reverse=True)
relevant_features_price_increase = sorted( ((v,k) for k,v in booster_price_increase.get_score().items()), reverse=True)
relevant_features_price_decrease = sorted( ((v,k) for k,v in booster_price_decrease.get_score().items()), reverse=True)
relevant_features_price_stagnation = sorted( ((v,k) for k,v in booster_price_stagnation.get_score().items()), reverse=True)

display(relevant_features_swing_trade)
display(relevant_features_price_increase)

In [ ]:
StockPrices['PredictSwingTradeScore'] = StockPrices.apply(lambda x:predict_score(x, booster_swing_trade, StockPrices, astro_columns), axis =1)
StockPrices['PredictPriceIncreaseScore'] = StockPrices.apply(lambda x:predict_score(x, booster_price_increase, StockPrices, astro_columns), axis =1)
StockPrices['PredictPriceDecreaseScore'] = StockPrices.apply(lambda x:predict_score(x, booster_price_decrease, StockPrices, astro_columns), axis =1)
StockPrices['PredictPriceStagnation'] = StockPrices.apply(lambda x:predict_score(x, booster_price_stagnation, StockPrices, astro_columns), axis =1)

In [ ]:
booster_swing_trade.save_model('./output/{}_swing_trade.model'.format(ASSET_TO_CALCULATE))
booster_price_increase.save_model('./output/{}_price_increase.model'.format(ASSET_TO_CALCULATE))
booster_price_decrease.save_model('./output/{}_price_decrease.model'.format(ASSET_TO_CALCULATE))
booster_price_stagnation.save_model('./output/{}_price_stagnation.model'.format(ASSET_TO_CALCULATE))

In [ ]:
swing_to_chart = []
for index, current_swing in StockPrices[StockPrices['PredictSwingTradeScore'] > 0.9].iterrows():
    swing_to_chart.append(dict(
        x0=current_swing['CorrectedDate'], 
        x1=current_swing['CorrectedDate'], 
        y0=0, 
        y1=1, 
        xref='x', 
        yref='paper',
        line_width=2))

In [ ]:
fig = go.Figure(data=[go.Candlestick(
                x=StockPrices['CorrectedDate'],
                open=StockPrices['Open'],
                high=StockPrices['High'],
                low=StockPrices['Low'],
                close=StockPrices['Price'])])
fig.update_layout(
    title="{} Swing Trade Opportunities detected by XGBoost".format(ASSET_TO_CALCULATE),
    width=1000,
    height=500,
    xaxis_rangeslider_visible=False,
    shapes=swing_to_chart,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=30,
        pad=4
    ),    
)
fig.show()

In [ ]:
swing_to_chart = []
for index, current_swing in StockPrices[StockPrices['PredictPriceIncreaseScore'] > 5].iterrows():
    swing_to_chart.append(dict(
        x0=current_swing['CorrectedDate'], 
        x1=current_swing['CorrectedDate'], 
        y0=0, 
        y1=1, 
        xref='x', 
        yref='paper',
        line_width=2))

In [ ]:
fig = go.Figure(data=[go.Candlestick(
                x=StockPrices['CorrectedDate'],
                open=StockPrices['Open'],
                high=StockPrices['High'],
                low=StockPrices['Low'],
                close=StockPrices['Price'])])
fig.update_layout(
    title="{} Price Increase Opportunities detected by XGBoost (Min {}%)".format(ASSET_TO_CALCULATE, STAGNATION_THRESHOLD),
    width=1000,
    height=500,
    xaxis_rangeslider_visible=False,
    shapes=swing_to_chart,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=30,
        pad=4
    ),    
)
fig.show()

In [ ]:
output_excel_file='./output/{}.Analisys.xlsx'.format(ASSET_TO_CALCULATE)
StockPrices.to_excel(output_excel_file)